In [9]:
import os
import pandas as pd

from WebScraper import scrape_url

In [10]:
base_url = "https://www.worldometers.info"

world_gdp_table = await scrape_url("https://www.worldometers.info/gdp/", ".datatable-container")
gdp_country_table = await scrape_url("https://www.worldometers.info/gdp/gdp-by-country/", ".datatable-container")

Navigating to: https://www.worldometers.info/gdp/
Waiting for selector: .datatable-container
Simulating human behavior...
Extracting page content...
Successfully extracted content from https://www.worldometers.info/gdp/
Navigating to: https://www.worldometers.info/gdp/gdp-by-country/
Waiting for selector: .datatable-container
Simulating human behavior...
Extracting page content...
Successfully extracted content from https://www.worldometers.info/gdp/gdp-by-country/


In [11]:
rows = []
table = world_gdp_table.find_all('table', class_="datatable")[0]

for tr in table.find_all("tr"):
    cells = tr.find_all(["td", "th"])
    row = [cell.get_text(strip=True) for cell in cells]
    rows.append(row)

gdp_world_df = pd.DataFrame(rows[1:], columns=rows[0])

gdp_world_df = gdp_world_df.rename(columns={
    'Year': 'Year',
    'GDP Real(Inflation adj.)': 'GDP_Inflation_Adjust',
    'GDPGrowth': "GDP_Growth",
    "PerCapita": "Per_Capita",
    "GDP Nominal(Current USD)": "GDP_Nominal_USD",
    "Pop.Change": "Population_Change",
    "WorldPopulation": "World_Population",
})
gdp_world_df

,Year,GDP_Inflation_Adjust,GDP_Growth,Per_Capita,GDP_Nominal_USD,Population_Change,World_Population
0,2023,"$93,346,700,000,000",2.83%,"$11,536","$106,172,000,000,000",0.88%,"8,091,734,930"
1,2022,"$90,774,600,000,000",3.24%,"$11,317","$101,771,000,000,000",0.84%,"8,021,407,192"
2,2021,"$87,927,500,000,000",6.35%,"$11,054","$97,848,300,000,000",0.86%,"7,954,448,391"
3,2020,"$82,677,400,000,000",−2.88%,"$10,483","$85,763,000,000,000",0.97%,"7,887,001,292"
4,2019,"$85,127,600,000,000",2.68%,"$10,898","$88,149,900,000,000",1.05%,"7,811,293,698"
...,...,...,...,...,...,...,...
58,1965,"$14,326,600,000,000",5.61%,"$4,296","$1,994,300,000,000",2.15%,"3,334,533,703"
59,1964,"$13,565,500,000,000",6.57%,"$4,155","$1,830,170,000,000",2.25%,"3,264,487,339"
60,1963,"$12,729,200,000,000",5.02%,"$3,987","$1,669,570,000,000",2.22%,"3,192,807,828"
61,1962,"$12,121,200,000,000",5.32%,"$3,881","$1,550,600,000,000",1.91%,"3,123,374,315"


In [12]:
rows = []
table = world_gdp_table.find_all('table', class_="datatable")[1]

for tr in table.find_all("tr"):
    cells = tr.find_all(["td", "th"])
    row = [cell.get_text(strip=True) for cell in cells]
    rows.append(row)

gdp_continent_df = pd.DataFrame(rows[1:], columns=rows[0])

gdp_continent_df = gdp_continent_df.rename(columns={
    'Region': 'Region',
    'GDP(nominal, 2023)': 'GDP_Nominal',
    'GDPGrowth': "GDP_Growth",
    "Share ofWorldGDP": "Share_World_GDP",
})
gdp_continent_df

,Region,GDP_Nominal,GDP_Growth,Share ofWorld GDP
0,Asia,"$37,528,310,623,669",4.34%,35.35%
1,Northern America,"$29,863,170,000,000",2.76%,28.13%
2,Europe,"$25,787,307,724,450",0.68%,24.29%
3,Latin America and the Caribbean,"$6,465,209,163,887",2.11%,6.09%
4,Africa,"$2,907,648,995,839",3.17%,2.74%
5,Oceania,"$2,021,447,749,966",3.13%,1.90%


In [18]:
rows = []

for i, tr in enumerate(gdp_country_table.find('table').find_all("tr")):
    cells = tr.find_all(["td", "th"])

    if i == 0:
        row = []
        for cell in cells:
            text = cell.get_text(strip=True)
            row.append(text)
            if "Country" in text:
                row.append("Link")
    else:
        row = []
        for j, cell in enumerate(cells):
            text = cell.get_text(strip=True)
            a_tag = cell.find("a")
            if j == 1 and a_tag and a_tag.get("href"):
                row.append(text)
                row.append(base_url + a_tag["href"])
            else:
                row.append(text)

    rows.append(row)

gdp_country_df = pd.DataFrame(rows[1:], columns=rows[0])

# Rename columns
gdp_country_df = gdp_country_df.rename(columns={
    '#': 'Rank',
    'Country (or dependency)': 'Country',
    'Link': 'Link',
    'GDP(nominal, 2023)': 'GDP_Nominal',
    'GDP(abbrev.)': "GDP_Abbrev",
    "GDPGrowth": "GDP_Growth",
    "Population(2023)": "Population",
    "GDPpercapita": "GDP_Per_Capita",
    "Share ofWorld GDP": "Share_World_GDP",
})

gdp_country_df

,Rank,Country,Link,GDP_Nominal,GDP_Abbrev,GDP_Growth,Population,GDP_Per_Capita,Share_World_GDP
0,1,United States,https://www.worldometers.info/gdp/us-gdp/,"$27,720,700,000,000",$27.721 trillion,2.89%,"343,477,335","$80,706",26.11%
1,2,China,https://www.worldometers.info/gdp/china-gdp/,"$17,794,800,000,000",$17.795 trillion,5.25%,"1,422,584,933","$12,509",16.76%
2,3,Germany,https://www.worldometers.info/gdp/germany-gdp/,"$4,525,700,000,000",$4.526 trillion,−0.27%,"84,548,231","$53,528",4.26%
3,4,Japan,https://www.worldometers.info/gdp/japan-gdp/,"$4,204,490,000,000",$4.204 trillion,1.68%,"124,370,947","$33,806",3.96%
4,5,India,https://www.worldometers.info/gdp/india-gdp/,"$3,567,550,000,000",$3.568 trillion,8.15%,"1,438,069,596","$2,481",3.36%
...,...,...,...,...,...,...,...,...,...
176,177,Micronesia,https://www.worldometers.info/gdp/micronesia-gdp/,"$460,000,000",$460 million,0.78%,"112,630","$4,084",0.00043%
177,178,Palau,https://www.worldometers.info/gdp/palau-gdp/,"$281,849,063",$281.849 million,1.88%,"17,727","$15,899",0.00027%
178,179,Kiribati,https://www.worldometers.info/gdp/kiribati-gdp/,"$279,208,903",$279.209 million,4.12%,"132,530","$2,107",0.00026%
179,180,Marshall Islands,https://www.worldometers.info/gdp/marshall-isl...,"$259,300,000",$259.3 million,−3.93%,"38,827","$6,678",0.00024%


In [19]:
for i, row in gdp_country_df[0:1].iterrows():
    link = row['Link']
    country = row['Country']

    country_population_soup = await scrape_url(link, ".datatable-table")

    rows = []
    table = country_population_soup.find_all('table', class_="datatable")

    for tr in table[0].find_all("tr"):
        cells = tr.find_all(["td", "th"])
        row = [cell.get_text(strip=True) for cell in cells]
        rows.append(row)

    df = pd.DataFrame(rows[1:], columns=rows[0])
    df = df.rename(columns={
            'Year': 'Year',
            'GDP Nominal(Current USD)': 'GDP_Nominal',
            'GDP Real(Inflation adj.)': 'GDP_Real_Inflation_Adjust',
            'GDPChange': 'GDP_Change',
            'GDPpercapita': "GDP_per_capita",
            "Pop.Change": "Population_Change",
        })
    
    print(df)


Navigating to: https://www.worldometers.info/gdp/us-gdp/
Waiting for selector: .datatable-table
Simulating human behavior...
Extracting page content...
Successfully extracted content from https://www.worldometers.info/gdp/us-gdp/
    Year          GDP_Nominal GDP_Real_Inflation_Adjust GDP_Change  \
0   2023  $27,720,700,000,000       $22,062,600,000,000      2.89%   
1   2022  $26,006,900,000,000       $21,443,400,000,000      2.51%   
2   2021  $23,681,200,000,000       $20,917,900,000,000      6.05%   
3   2020  $21,354,100,000,000       $19,723,600,000,000     −2.16%   
4   2019  $21,540,000,000,000       $20,159,600,000,000      2.58%   
5   2018  $20,656,500,000,000       $19,651,900,000,000      2.97%   
6   2017  $19,612,100,000,000       $19,085,700,000,000      2.46%   
7   2016  $18,804,900,000,000       $18,627,900,000,000      1.82%   
8   2015  $18,295,000,000,000       $18,295,000,000,000      2.95%   
9   2014  $17,608,100,000,000       $17,771,500,000,000      2.52%   
